In [ ]:
import os
import json
import gzip
import pandas as pd
from urllib.request import urlopen

In [ ]:
from google.colab import drive

drive.mount('/content/drive') 

Mounted at /content/drive


In [ ]:
%cd '/content/drive/My Drive/FSI_Problem'
!ls

/content/drive/My Drive/FSI_Problem
 Bert_Sentiment_AmazonDataset.ipynb	   PhoBert_Sentiment.ipynb
 check_text.csv				   svc_rbf.pkl
 Clone_Foody.ipynb			   test.crash
 Concat_Data.ipynb			   Test_ML.ipynb
'Copy of Test_Foody.ipynb'		  'Test_ML with Tokenize.ipynb'
'Copy of Viblo_Sentiment.ipynb'		   Test_PhoBert.ipynb
 Data					   tfidf.pkl
 LR_Model.pkl				   tfidf_rbf.pkl
 MultiBert_Sentiment_AmazonDataset.ipynb   train.crash
 PhoBERT_base_fairseq			   train_dataloader.pkl
 PhoBERT_base_fairseq.tar.gz		   val_dataloader.pkl
 PhoBERT_base_transformers		   vncorenlp
 PhoBERT_base_transformers.tar.gz	   X1.pkl
 PhoBert_Sentiment_Foody.ipynb		   y1.pkl


In [ ]:
data = pd.read_csv('Data/datadepvail.csv') 
data = data.drop(['Unnamed: 0'],axis=1)
data.info() 

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9136 entries, 0 to 9135
Data columns (total 2 columns):
 #   Column   Non-Null Count  Dtype  
---  ------   --------------  -----  
 0   comment  9132 non-null   object 
 1   labels   9128 non-null   float64
dtypes: float64(1), object(1)
memory usage: 142.9+ KB


In [ ]:
data = data.dropna()  
data['labels'].value_counts()

0.0    3925
2.0    3125
1.0    2074
Name: labels, dtype: int64

In [ ]:
data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 9124 entries, 0 to 9135
Data columns (total 2 columns):
 #   Column   Non-Null Count  Dtype  
---  ------   --------------  -----  
 0   comment  9124 non-null   object 
 1   labels   9124 non-null   float64
dtypes: float64(1), object(1)
memory usage: 213.8+ KB


In [ ]:
a =list(data[data['label'] ==0].index)
b = []
for i in range(0,800):
  b.append(a[i]) 

In [ ]:
data= data.drop(b)
data['label'].value_counts()

1.0    743
0.0    716
2.0    681
Name: label, dtype: int64

In [ ]:
data.to_csv('Data/data_check.csv')

In [ ]:
X = data.comment.values
y = data.labels.values

In [ ]:
from sklearn.model_selection import train_test_split

X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.15,random_state = 7)

In [ ]:
print(sum(y_train==0))
print(sum(y_train==1))
print(sum(y_train==2))

3366
1735
2654


In [ ]:
import torch

if torch.cuda.is_available():       
    device = torch.device("cuda")
    print(f'There are {torch.cuda.device_count()} GPU(s) available.')
    print('Device name:', torch.cuda.get_device_name(0))

else:
    print('No GPU available, using the CPU instead.')
    device = torch.device("cpu")

There are 1 GPU(s) available.
Device name: Tesla T4


In [ ]:
!pip3 install fairseq
!pip3 install fastbpe
!pip3 install vncorenlp
!pip3 install transformers

     |████████████████████████████████| 686kB 17.6MB/s 
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Installing backend dependencies ... done
    Preparing wheel metadata ... done
     |████████████████████████████████| 71kB 10.6MB/s 
     |████████████████████████████████| 122kB 58.5MB/s 
     |████████████████████████████████| 112kB 57.7MB/s 
     |████████████████████████████████| 276kB 50.9MB/s 
  Created wheel for fairseq: filename=fairseq-0.10.0-cp36-cp36m-linux_x86_64.whl size=2632011 sha256=0ebd964f5abea7f43cc1a9fb0e73cdd7483c2f0e9ade619cbd7d1bd2c8e1d34a
  Stored in directory: /root/.cache/pip/wheels/cc/23/f1/b5dba0d2da81577b3c426ef983e4bc021e5f751f35c5ba94a2
Successfully built fairseq
  Created wheel for antlr4-python3-runtime: filename=antlr4_python3_runtime-4.8-cp36-none-any.whl size=141230 sha256=3ca1d397e7cfcb10e0e45f5e446a2832a295929c253b49ce5aaf680fb1230966
  Stored in directory: /root/.cache/pip/wheels/e3/e2/fa/b78480b448b857

In [ ]:
from vncorenlp import VnCoreNLP
rdrsegmenter = VnCoreNLP("/content/drive/My Drive/FSI_Problem/vncorenlp/VnCoreNLP-1.1.1.jar", annotators="wseg", max_heap_size='-Xmx500m') 

text = "Đại học Bách Khoa Hà Nội."

word_segmented_text = rdrsegmenter.tokenize(text) 
print(word_segmented_text)

[['Đại_học', 'Bách_Khoa', 'Hà_Nội', '.']]


In [ ]:
from fairseq.data.encoders.fastbpe import fastBPE
from fairseq.data import Dictionary
import argparse

parser = argparse.ArgumentParser()
parser.add_argument('--bpe-codes', 
    default="/content/drive/My Drive/FSI_Problem/PhoBERT_base_transformers/bpe.codes",
    required=False,
    type=str,
    help='path to fastBPE BPE'
)
args, unknown = parser.parse_known_args()
bpe = fastBPE(args)

# Load the dictionary
vocab = Dictionary()
vocab.add_from_file("/content/drive/My Drive/FSI_Problem/PhoBERT_base_transformers/dict.txt")

In [ ]:
vocab.encode_line('<s> ' + 'Hôm_nay trời nóng quá nên tôi ở nhà viết Vi@@ blo@@ !' )

tensor([    0,  3791,  1027,   898,   204,    77,    70,    25,    69,   467,
         3696, 16856,   381,     2], dtype=torch.int32)

In [ ]:
import re
def text_preprocessing(text):

    text = re.sub('[^a-zA-ZÀÁÂÃÈÉÊÌÍÒÓÔÕÙÚÝàáâãèéêìíòóôõùúýĂăĐđĨĩŨũƠơƯưẠạẢảẤấẦầẨẩẪẫẬậẮắẰằẲẳẴẵẶặẸẹẺẻẼẽẾếỀềỂểỄễỆệỈỉỊịỌọỎỏỐốỒồỔổỖỗỘộỚớỜờỞởỠỡỢợỤụỦủỨứỪừỬửỮữỰựỲỳỴỵỶỷỸỹ0-9.,"-]', ' ' ,text)

    text = re.sub(r'\s+', ' ', text).strip()

    return text

In [ ]:
sample = X[20]
sample = text_preprocessing(sample)
splits = sample.strip().split(' ')
text = ' '.join(splits)
text = rdrsegmenter.tokenize(text)
text = ' '.join([' '.join(x) for x in text])
print(text)

rất là đẹp


In [ ]:
y[20]

2.0

In [ ]:
import re
train_id, train_text, train_labels = [], [], []
val_id, val_text, val_labels = [], [], []

for sample in X_train:
  sample = text_preprocessing(sample)
  
  splits = sample.strip().split(' ')
  text = ' '.join(splits)
  text = rdrsegmenter.tokenize(text)
  text = ' '.join([' '.join(x) for x in text])

  train_text.append(text) 

for sample in X_val:
  sample = text_preprocessing(sample)
  
  splits = sample.strip().split(' ')
  text = ' '.join(splits)
  text = rdrsegmenter.tokenize(text)
  text = ' '.join([' '.join(x) for x in text])

  val_text.append(text) 

In [ ]:
from tensorflow.keras.preprocessing.sequence import pad_sequences
MAX_LEN = 250

train_ids = []
for sent in train_text:
    subwords = '<s> ' + bpe.encode(sent)
    encoded_sent = vocab.encode_line(subwords, append_eos=True, add_if_not_exist=False).long().tolist()
    train_ids.append(encoded_sent)

val_ids = []
for sent in val_text:
    subwords = '<s> ' + bpe.encode(sent)
    encoded_sent = vocab.encode_line(subwords, append_eos=True, add_if_not_exist=False).long().tolist()
    val_ids.append(encoded_sent)
    
train_ids = pad_sequences(train_ids, maxlen=MAX_LEN, dtype="long", value=0, truncating="post", padding="post")
val_ids = pad_sequences(val_ids, maxlen=MAX_LEN, dtype="long", value=0, truncating="post", padding="post")

In [ ]:
train_masks = []
for sent in train_ids:
    mask = [int(token_id > 0) for token_id in sent]
    train_masks.append(mask)

val_masks = []
for sent in val_ids:
    mask = [int(token_id > 0) for token_id in sent]
    val_masks.append(mask)

In [ ]:
print(train_ids[2])
train_text[2]

[    0    74    10    36   192 13304    51   265  2529   741     2     0
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0   

'vẫn có nhiều điểm bất_tiện nhưng sản_phẩm đa chức_năng'

In [ ]:
from transformers import BertModel, RobertaConfig, AdamW

config = RobertaConfig.from_pretrained(
    "/content/drive/My Drive/FSI_Problem/PhoBERT_base_transformers/config.json", from_tf=False, num_labels = 2, output_hidden_states=False,
)
BERT_SA = BertModel.from_pretrained(
    "/content/drive/My Drive/FSI_Problem/PhoBERT_base_transformers/model.bin",
    config=config
) 

Some weights of the model checkpoint at /content/drive/My Drive/FSI_Problem/PhoBERT_base_transformers/model.bin were not used when initializing BertModel: ['roberta.embeddings.word_embeddings.weight', 'roberta.embeddings.position_embeddings.weight', 'roberta.embeddings.token_type_embeddings.weight', 'roberta.embeddings.LayerNorm.weight', 'roberta.embeddings.LayerNorm.bias', 'roberta.encoder.layer.0.attention.self.query.weight', 'roberta.encoder.layer.0.attention.self.query.bias', 'roberta.encoder.layer.0.attention.self.key.weight', 'roberta.encoder.layer.0.attention.self.key.bias', 'roberta.encoder.layer.0.attention.self.value.weight', 'roberta.encoder.layer.0.attention.self.value.bias', 'roberta.encoder.layer.0.attention.output.dense.weight', 'roberta.encoder.layer.0.attention.output.dense.bias', 'roberta.encoder.layer.0.attention.output.LayerNorm.weight', 'roberta.encoder.layer.0.attention.output.LayerNorm.bias', 'roberta.encoder.layer.0.intermediate.dense.weight', 'roberta.encoder.l

In [ ]:
from torch.utils.data import TensorDataset, DataLoader, RandomSampler, SequentialSampler
import torch
train_inputs = torch.LongTensor(train_ids)
val_inputs = torch.LongTensor(val_ids)
y_train = torch.tensor(y_train)
y_train = y_train.type(torch.LongTensor)
y_val = torch.tensor(y_val)
y_val = y_val.type(torch.LongTensor)
train_masks = torch.LongTensor(train_masks)
val_masks = torch.LongTensor(val_masks)

train_data = TensorDataset(train_inputs, train_masks, y_train)
train_sampler = SequentialSampler(train_data)
train_dataloader = DataLoader(train_data, sampler=train_sampler, batch_size=32)

val_data = TensorDataset(val_inputs, val_masks, y_val)
val_sampler = SequentialSampler(val_data)
val_dataloader = DataLoader(val_data, sampler=val_sampler, batch_size=32)

In [ ]:
%%time
import torch
import torch.nn as nn
from transformers import BertModel

# Create the BertClassfier class
class BertClassifier(nn.Module):
    """Bert Model for Classification Tasks.
    """
    def __init__(self, freeze_bert=False):
        """
        @param    bert: a BertModel object
        @param    classifier: a torch.nn.Module classifier
        @param    freeze_bert (bool): Set `False` to fine-tune the BERT model
        """
        super(BertClassifier, self).__init__()
        # Specify hidden size of BERT, hidden size of our classifier, and number of labels
        D_in, H, D_out = 768, 50, 3

        # Instantiate BERT model
        self.bert = BERT_SA

        # Instantiate an one-layer feed-forward classifier
        self.classifier = nn.Sequential(
            nn.Linear(D_in, H),
            nn.ReLU(),
            nn.Dropout(0.5),
            nn.Linear(H, D_out)
        )

        # Freeze the BERT model
        if freeze_bert:
            for param in self.bert.parameters():
                param.requires_grad = False
        
    def forward(self, input_ids, attention_mask):
        """
        Feed input to BERT and the classifier to compute logits.
        @param    input_ids (torch.Tensor): an input tensor with shape (batch_size,
                      max_length)
        @param    attention_mask (torch.Tensor): a tensor that hold attention mask
                      information with shape (batch_size, max_length)
        @return   logits (torch.Tensor): an output tensor with shape (batch_size,
                      num_labels)
        """
        # Feed input to BERT
        outputs = self.bert(input_ids=input_ids,
                            attention_mask=attention_mask)
        
        # Extract the last hidden state of the token `[CLS]` for classification task
        last_hidden_state_cls = outputs[0][:, 0, :]

        # Feed input to classifier to compute logits
        logits = self.classifier(last_hidden_state_cls)
        
        return logits

CPU times: user 44 µs, sys: 0 ns, total: 44 µs
Wall time: 46.3 µs


In [ ]:
from transformers import AdamW, get_linear_schedule_with_warmup

def initialize_model(epochs=4):
    """Initialize the Bert Classifier, the optimizer and the learning rate scheduler.
    """
    # Instantiate Bert Classifier
    bert_classifier = BertClassifier(freeze_bert=False)

    bert_classifier.to(device)
    
    optimizer = AdamW(bert_classifier.parameters(),
                      lr=5e-5,    # Default learning rate
                      eps=1e-8    # Default epsilon value
                      )

    # Total number of training steps
    total_steps = len(train_dataloader) * epochs

    # Set up the learning rate scheduler
    scheduler = get_linear_schedule_with_warmup(optimizer,
                                                num_warmup_steps=0, # Default value
                                                num_training_steps=total_steps)
    return bert_classifier, optimizer, scheduler


In [ ]:
import numpy as np
import random
def set_seed(seed_value=42):
    """Set seed for reproducibility.
    """
    random.seed(seed_value)
    np.random.seed(seed_value)
    torch.manual_seed(seed_value)
    torch.cuda.manual_seed_all(seed_value)

In [ ]:
set_seed(42)    # Set seed for reproducibility
bert_classifier, optimizer, scheduler = initialize_model(epochs=4)

In [ ]:
for n,p in bert_classifier.named_parameters():
  print(p.device, ' ' , n)

cuda:0   bert.embeddings.word_embeddings.weight
cuda:0   bert.embeddings.position_embeddings.weight
cuda:0   bert.embeddings.token_type_embeddings.weight
cuda:0   bert.embeddings.LayerNorm.weight
cuda:0   bert.embeddings.LayerNorm.bias
cuda:0   bert.encoder.layer.0.attention.self.query.weight
cuda:0   bert.encoder.layer.0.attention.self.query.bias
cuda:0   bert.encoder.layer.0.attention.self.key.weight
cuda:0   bert.encoder.layer.0.attention.self.key.bias
cuda:0   bert.encoder.layer.0.attention.self.value.weight
cuda:0   bert.encoder.layer.0.attention.self.value.bias
cuda:0   bert.encoder.layer.0.attention.output.dense.weight
cuda:0   bert.encoder.layer.0.attention.output.dense.bias
cuda:0   bert.encoder.layer.0.attention.output.LayerNorm.weight
cuda:0   bert.encoder.layer.0.attention.output.LayerNorm.bias
cuda:0   bert.encoder.layer.0.intermediate.dense.weight
cuda:0   bert.encoder.layer.0.intermediate.dense.bias
cuda:0   bert.encoder.layer.0.output.dense.weight
cuda:0   bert.encoder.

In [ ]:
a = list(bert_classifier.named_parameters())
print(a[-4])

('classifier.0.weight', Parameter containing:
tensor([[ 0.0276,  0.0300, -0.0085,  ...,  0.0203,  0.0096, -0.0338],
        [-0.0232,  0.0357,  0.0138,  ..., -0.0029, -0.0035, -0.0226],
        [ 0.0053, -0.0111, -0.0186,  ...,  0.0005, -0.0162, -0.0285],
        ...,
        [ 0.0269,  0.0332, -0.0217,  ...,  0.0001, -0.0270,  0.0292],
        [-0.0131, -0.0277,  0.0359,  ...,  0.0158,  0.0274, -0.0046],
        [-0.0348, -0.0182, -0.0180,  ..., -0.0194,  0.0054,  0.0325]],
       device='cuda:0', requires_grad=True))


In [ ]:
import random
import time

# Specify loss function
loss_fn = nn.CrossEntropyLoss()

def set_seed(seed_value=42):
    """Set seed for reproducibility.
    """
    random.seed(seed_value)
    np.random.seed(seed_value)
    torch.manual_seed(seed_value)
    torch.cuda.manual_seed_all(seed_value)

def train(model, train_dataloader, val_dataloader=None, epochs=4, evaluation=False):
    """Train the BertClassifier model.
    """
    # Start training loop
    print("Start training...\n")
    for epoch_i in range(epochs):
        # =======================================
        #               Training
        # =======================================
        # Print the header of the result table
        print(f"{'Epoch':^7} | {'Batch':^7} | {'Train Loss':^12} | {'Val Loss':^10} | {'Val Acc':^9} | {'Elapsed':^9}")
        print("-"*70)

        # Measure the elapsed time of each epoch
        t0_epoch, t0_batch = time.time(), time.time()

        # Reset tracking variables at the beginning of each epoch
        total_loss, batch_loss, batch_counts = 0, 0, 0

        # Put the model into the training mode
        model.train()

        # For each batch of training data...
        for step, batch in enumerate(train_dataloader):
            batch_counts +=1
            # Load batch to GPU
            b_input_ids, b_attn_mask, b_labels = tuple(t.to(device) for t in batch)

            # Zero out any previously calculated gradients
            model.zero_grad()

            # Perform a forward pass. This will return logits.
            logits = model(b_input_ids, b_attn_mask)

            # Compute loss and accumulate the loss values
            loss = loss_fn(logits, b_labels)
            batch_loss += loss.item()
            total_loss += loss.item()

            # Perform a backward pass to calculate gradients
            loss.backward()

            # Clip the norm of the gradients to 1.0 to prevent "exploding gradients"
            torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)

            # Update parameters and the learning rate
            optimizer.step()
            scheduler.step()

            # Print the loss values and time elapsed for every 20 batches
            if (step % 20 == 0 and step != 0) or (step == len(train_dataloader) - 1):
                # Calculate time elapsed for 20 batches
                time_elapsed = time.time() - t0_batch

                # Print training results
                print(f"{epoch_i + 1:^7} | {step:^7} | {batch_loss / batch_counts:^12.6f} | {'-':^10} | {'-':^9} | {time_elapsed:^9.2f}")

                # Reset batch tracking variables
                batch_loss, batch_counts = 0, 0
                t0_batch = time.time()

        # Calculate the average loss over the entire training data
        avg_train_loss = total_loss / len(train_dataloader)

        print("-"*70)
        # =======================================
        #               Evaluation
        # =======================================
        if evaluation == True:
            # After the completion of each training epoch, measure the model's performance
            # on our validation set.
            val_loss, val_accuracy = evaluate(model, val_dataloader)

            # Print performance over the entire training data
            time_elapsed = time.time() - t0_epoch
            
            print(f"{epoch_i + 1:^7} | {'-':^7} | {avg_train_loss:^12.6f} | {val_loss:^10.6f} | {val_accuracy:^9.2f} | {time_elapsed:^9.2f}")
            print("-"*70)
        print("\n")
    
    print("Training complete!")


def evaluate(model, val_dataloader):
    """After the completion of each training epoch, measure the model's performance
    on our validation set.
    """
    # Put the model into the evaluation mode. The dropout layers are disabled during
    # the test time.
    model.eval()

    # Tracking variables
    val_accuracy = []
    val_loss = []

    # For each batch in our validation set...
    for batch in val_dataloader:
        # Load batch to GPU
        b_input_ids, b_attn_mask, b_labels = tuple(t.to(device) for t in batch)

        # Compute logits
        with torch.no_grad():
            logits = model(b_input_ids, b_attn_mask)

        # Compute loss
        loss = loss_fn(logits, b_labels)
        val_loss.append(loss.item())

        # Get the predictions
        preds = torch.argmax(logits, dim=1).flatten()

        # Calculate the accuracy rate
        accuracy = (preds == b_labels).cpu().numpy().mean() * 100
        val_accuracy.append(accuracy)

    # Compute the average accuracy and loss over the validation set.
    val_loss = np.mean(val_loss)
    val_accuracy = np.mean(val_accuracy)

    return val_loss, val_accuracy

In [ ]:
train(bert_classifier, train_dataloader, val_dataloader, epochs=4, evaluation=True)

Start training...

 Epoch  |  Batch  |  Train Loss  |  Val Loss  |  Val Acc  |  Elapsed 
----------------------------------------------------------------------
   1    |   20    |   1.129122   |     -      |     -     |   25.42  
   1    |   40    |   1.105973   |     -      |     -     |   24.53  
   1    |   60    |   1.076003   |     -      |     -     |   25.05  
   1    |   80    |   1.090855   |     -      |     -     |   25.38  
   1    |   100   |   1.084074   |     -      |     -     |   25.89  
   1    |   120   |   1.077338   |     -      |     -     |   26.16  
   1    |   140   |   1.081648   |     -      |     -     |   26.47  
   1    |   160   |   1.084343   |     -      |     -     |   26.59  
   1    |   180   |   1.071438   |     -      |     -     |   26.82  
   1    |   200   |   1.057792   |     -      |     -     |   27.09  
   1    |   220   |   1.070323   |     -      |     -     |   27.27  
   1    |   240   |   1.071327   |     -      |     -     |   27.27  


KeyboardInterrupt: ignored

In [ ]:
import torch.nn.functional as F

def bert_predict(model, test_dataloader):
    """Perform a forward pass on the trained BERT model to predict probabilities
    on the test set.
    """
    # Put the model into the evaluation mode. The dropout layers are disabled during
    # the test time.
    model.eval()

    all_logits = []

    # For each batch in our test set...
    for batch in test_dataloader:
        # Load batch to GPU
        b_input_ids, b_attn_mask = tuple(t.to(device) for t in batch)[:2]

        # Compute logits
        with torch.no_grad():
            logits = model(b_input_ids, b_attn_mask)
        all_logits.append(logits)
    
    # Concatenate logits from each batch
    all_logits = torch.cat(all_logits, dim=0)

    # Apply softmax to calculate probabilities
    probs = F.softmax(all_logits, dim=1).cpu().numpy()
    
    return probs 

In [ ]:
probs = bert_predict(bert_classifier, val_dataloader) 


In [ ]:
probs_index = []
for i in probs:
  probs_index.append(np.argmax(i))

probs_index = np.array(probs_index)
probs_index

array([1, 1, 1, ..., 1, 1, 1])

In [ ]:
y_val = np.array(y_val)

In [ ]:
print(sum(probs_index==0))
print(sum(probs_index==1))
print(sum(probs_index==2))
print(sum(y_val!=probs_index))

0
2165
0
1409
